In [ ]:
import os
import sys

import h5py
import numpy as np
import seaborn as sns

sys.path.insert(0, os.path.abspath("../.."))

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.collections import LineCollection

from transformers.data_utils.bb_data_loading import load_list
from transformers.training.jax_utils import cross_ent_loss
from transformers.training.utils import load_pickle

In [ ]:
t0012_list = load_list("../t0012/preference_data_1/day_list.txt")

In [ ]:
t0012_day = np.random.choice(t0012_list)
with h5py.File(f"../t0012/preference_data_1/{t0012_day}.hdf5") as f:
    r_idx = np.random.choice(f["states_2"].shape[0])
    sts_2 = f["states_2"][r_idx, ...].reshape(1, 100, 16)
    acts_2 = f["actions_2"][r_idx, ...].reshape(1, 100, 3)
    ts_2 = f["timesteps_2"][r_idx, ...].reshape(1, 100)
    am_2 = f["attn_mask_2"][r_idx, ...].reshape(1, 100)

    sts = f["states"][r_idx, ...].reshape(1, 100, 16)
    acts = f["actions"][r_idx, ...].reshape(1, 100, 3)
    ts = f["timesteps"][r_idx, ...].reshape(1, 100)
    am = f["attn_mask"][r_idx, ...].reshape(1, 100)

In [ ]:
# with h5py.File("../t0012/preference_data_1/test.2022.01.16.09.00.26.hdf5") as f:
#     sts_2 = f["states_2"][6, ...].reshape(1, 100, 16)
#     acts_2 = f["actions_2"][6, ...].reshape(1, 100, 3)
#     ts_2 = f["timesteps_2"][6, ...].reshape(1, 100)
#     am_2 = f["attn_mask_2"][6, ...].reshape(1, 100)

#     sts = f["states"][6, ...].reshape(1, 100, 16)
#     acts = f["actions"][6, ...].reshape(1, 100, 3)
#     ts = f["timesteps"][6, ...].reshape(1, 100)
#     am = f["attn_mask"][6, ...].reshape(1, 100)

In [ ]:
n_rewards_2 = []
n_all_weights_2 = []
n_max_reward_2 = -np.inf
n_min_reward_2 = np.inf

n_max_weight_2 = -np.inf
n_min_weight_2 = np.inf

all_rewards_2 = []

n_rewards = []
n_all_weights = []
n_max_reward = -np.inf
n_min_reward = np.inf

n_max_weight = -np.inf
n_min_weight = np.inf

all_rewards = []

pr = []
for i in range(len(t0012_list)):
    sts_2[:, :, 15] = np.where(am_2 != 0, i, sts_2[:, :, 15])
    sts[:, :, 15] = np.where(am != 0, i, sts[:, :, 15])
    n_model = load_pickle("../t0012/results/pt_rewards/best_model.pkl")["model"]
    n_pred_2, n_weights_2 = n_model._train_state.apply_fn(
        n_model._train_state.params, sts_2, acts_2, ts_2, training=False, attn_mask=am_2
    )
    n_pred, n_weights = n_model._train_state.apply_fn(
        n_model._train_state.params, sts, acts, ts, training=False, attn_mask=am
    )

    n_r_pred_2 = n_pred_2["value"].reshape(
        100,
    )

    n_r_pred = n_pred["value"].reshape(
        100,
    )

    trans_pred_1 = n_pred["weighted_sum"]
    trans_pred_2 = n_pred_2["weighted_sum"]

    sum_pred_1 = np.nanmean(trans_pred_1.reshape(1, 100), axis=1).reshape(-1, 1)
    sum_pred_2 = np.nanmean(trans_pred_2.reshape(1, 100), axis=1).reshape(-1, 1)

    pr.append((1.0 / (np.exp(sum_pred_1 - sum_pred_2) + 1.0))[0][0])

    am_sum_2 = int(np.sum(am_2))
    n_r_pred_2 = n_r_pred_2[:am_sum_2]
    n_max_r_pred_2 = np.max(n_r_pred_2)
    if n_max_r_pred_2 > n_max_reward_2:
        n_max_reward_2 = n_max_r_pred_2

    n_min_r_pred_2 = np.min(n_r_pred_2)
    if n_min_r_pred_2 < n_min_reward_2:
        n_min_reward_2 = n_min_r_pred_2

    n_ts_2 = np.arange(am_sum_2)
    n_rewards_2.append(np.column_stack([n_ts_2, n_r_pred_2]))
    all_rewards_2.append(n_r_pred_2)
    n_weights_2 = n_weights_2[-1].reshape(1, 100, 100)
    n_weights_2 = np.mean(n_weights_2, axis=1).reshape(
        100,
    )
    n_weights_2 = n_weights_2[:am_sum_2]
    n_all_weights_2.append(np.column_stack([n_ts_2, n_weights_2]))
    n_max_w_2 = np.max(n_weights_2)
    if n_max_w_2 > n_max_weight_2:
        n_max_weight_2 = n_max_w_2

    am_sum = int(np.sum(am))
    n_r_pred = n_r_pred[:am_sum]
    n_max_r_pred = np.max(n_r_pred)
    if n_max_r_pred > n_max_reward:
        n_max_reward = n_max_r_pred

    n_min_r_pred = np.min(n_r_pred)
    if n_min_r_pred < n_min_reward:
        n_min_reward = n_min_r_pred

    n_ts = np.arange(am_sum)
    n_rewards.append(np.column_stack([n_ts, n_r_pred]))
    all_rewards.append(n_r_pred)
    n_weights = n_weights[-1].reshape(1, 100, 100)
    n_weights = np.mean(n_weights, axis=1).reshape(
        100,
    )
    n_weights = n_weights[:am_sum]
    n_all_weights.append(np.column_stack([n_ts, n_weights]))
    n_max_w = np.max(n_weights)
    if n_max_w > n_max_weight:
        n_max_weight = n_max_w

In [ ]:
start_control_idxs_2 = []
end_control_idxs_2 = []
for i in range(100):
    if acts_2[0, i, 2] == 1:
        if i == 0:
            start_control_idxs_2.append(i)
        else:
            if acts_2[0, i - 1, 2] == 0:
                start_control_idxs_2.append(i)
    else:
        if acts_2[0, i - 1, 2] == 1:
            end_control_idxs_2.append(i)

start_control_idxs = []
end_control_idxs = []
for i in range(100):
    if acts[0, i, 2] == 1:
        if i == 0:
            start_control_idxs.append(i)
        else:
            if acts[0, i - 1, 2] == 0:
                start_control_idxs.append(i)
    else:
        if acts[0, i - 1, 2] == 1:
            end_control_idxs.append(i)

In [ ]:
all_rewards_2 = np.stack(all_rewards_2)
norm_rewards_2 = (all_rewards_2 - np.min(all_rewards_2, axis=0)) / (
    np.max(all_rewards_2, axis=0) - np.min(all_rewards_2, axis=0)
)

all_rewards = np.stack(all_rewards)
norm_rewards = (all_rewards - np.min(all_rewards, axis=0)) / (
    np.max(all_rewards, axis=0) - np.min(all_rewards, axis=0)
)

In [ ]:
fig, axe = plt.subplots(3, 2, figsize=(15, 10))
plt.subplots_adjust(hspace=0.5, wspace=0.3)
axe[0, 0].set_xlim(0, am_sum_2)
axe[0, 0].set_ylim(n_min_reward_2, n_max_reward_2)
n_days_2 = np.arange(len(n_all_weights_2))
n_line_collection0_2 = LineCollection(n_rewards_2, array=n_days_2, cmap="rainbow")
axe[0, 0].add_collection(n_line_collection0_2)
axe[0, 0].vlines(
    start_control_idxs_2,
    n_min_reward_2,
    n_max_reward_2,
    color="green",
    linestyle="dashed",
    label="Start Human Control",
)
axe[0, 0].vlines(
    end_control_idxs_2,
    n_min_reward_2,
    n_max_reward_2,
    color="magenta",
    linestyle="dashed",
    label="End Human Control",
)
# axe[0,0].errorbar(np.arange(am_sum_2),np.mean(all_rewards_2,axis=0),yerr=np.std(all_rewards_2,axis=0),color="black",label="Mean")
# axe[0,0].plot(np.arange(am_sum_2),np.median(all_rewards_2,axis=0),color="green",label="Median")
axe[0, 0].set_title("Reward Signal (t0012)")
axe[0, 0].set_xlabel("Timestep")
axe[0, 0].set_ylabel("Reward")
axe[0, 0].legend(loc="upper left")
fig.colorbar(n_line_collection0_2, label="Day")

axe[1, 0].set_xlim(0, am_sum_2)
axe[1, 0].set_ylim(0, n_max_weight_2)
n_line_collection1_2 = LineCollection(n_all_weights_2, array=n_days_2, cmap="rainbow")
axe[1, 0].add_collection(n_line_collection1_2)
axe[1, 0].vlines(
    start_control_idxs_2,
    0,
    n_max_weight_2,
    color="green",
    linestyle="dashed",
    label="Start Human Control",
)
axe[1, 0].vlines(
    end_control_idxs_2,
    0,
    n_max_weight_2,
    color="magenta",
    linestyle="dashed",
    label="End Human Control",
)
axe[1, 0].set_title("Importance Weighting (t0012)")
axe[1, 0].set_xlabel("Timestep")
axe[1, 0].set_ylabel("Weight")
axe[1, 0].legend(loc="upper left")
fig.colorbar(n_line_collection1_2, label="Day")

sns.heatmap(norm_rewards_2, cmap="cool", ax=axe[2, 0], cbar_kws={"label": "Reward"})
axe[2, 0].invert_yaxis()
axe[2, 0].set_xlabel("Timestep")
axe[2, 0].set_ylabel("Day")
axe[2, 0].set_title("Relative Reward Signal Heatmap (t0012)")

axe[0, 1].set_xlim(0, am_sum)
axe[0, 1].set_ylim(n_min_reward, n_max_reward)
n_days = np.arange(len(n_all_weights))
n_line_collection0 = LineCollection(n_rewards, array=n_days, cmap="rainbow")
axe[0, 1].add_collection(n_line_collection0)
axe[0, 1].vlines(
    start_control_idxs,
    n_min_reward,
    n_max_reward,
    color="green",
    linestyle="dashed",
    label="Start Human Control",
)
axe[0, 1].vlines(
    end_control_idxs,
    n_min_reward,
    n_max_reward,
    color="magenta",
    linestyle="dashed",
    label="End Human Control",
)
# axe[0,1].errorbar(np.arange(am_sum),np.mean(all_rewards,axis=0),yerr=np.std(all_rewards,axis=0),color="black",label="Mean")
# axe[0,1].plot(np.arange(am_sum),np.median(all_rewards,axis=0),color="green",label="Median")
axe[0, 1].set_title("Reward Signal (t0012)")
axe[0, 1].set_xlabel("Timestep")
axe[0, 1].set_ylabel("Reward")
axe[0, 1].legend(loc="upper left")
fig.colorbar(n_line_collection0, label="Day")

axe[1, 1].set_xlim(0, am_sum)
axe[1, 1].set_ylim(0, n_max_weight)
n_line_collection1 = LineCollection(n_all_weights, array=n_days, cmap="rainbow")
axe[1, 1].add_collection(n_line_collection1)
axe[1, 1].vlines(
    start_control_idxs,
    0,
    n_max_weight,
    color="green",
    linestyle="dashed",
    label="Start Human Control",
)
axe[1, 1].vlines(
    end_control_idxs,
    0,
    n_max_weight,
    color="magenta",
    linestyle="dashed",
    label="End Human Control",
)
axe[1, 1].set_title("Importance Weighting (t0012)")
axe[1, 1].set_xlabel("Timestep")
axe[1, 1].set_ylabel("Weight")
axe[1, 1].legend(loc="upper left")
fig.colorbar(n_line_collection1, label="Day")

sns.heatmap(norm_rewards, cmap="cool", ax=axe[2, 1], cbar_kws={"label": "Reward"})
axe[2, 1].invert_yaxis()
axe[2, 1].set_xlabel("Timestep")
axe[2, 1].set_ylabel("Day")
axe[2, 1].set_title("Relative Reward Signal Heatmap (t0012)")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(pr)
ax.set_title("Preference Prediction (for segment 2) Versus Day")
ax.set_xlabel("Day")
ax.set_ylabel("Probability of Preferred Label")
plt.show()